In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import requests
import urllib.request
import time

authors = []
dates = []
statements = []
content = []
sources = []
targets = []


def scrape_website(page_number):
  pgno = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+ pgno + '&category=environment'
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text,'html.parser')
  #Get location
  statement_footer = soup.find_all('footer',attrs ={'class':'m-statement__footer'})
  statement_quote = soup.find_all('div',attrs={'class':'m-statement__quote'})
  statement_meta = soup.find_all('div',attrs={'class':'m-statement__meta'})
  target = soup.find_all('div',attrs={'class':'m-statement__meter'})

  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month + ' '+ day + ' '+year
    dates.append(date)
    authors.append(full_name)

  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)
    atag=i.find_all('a')[0]
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    URL2 = 'https://www.politifact.com' +  atag['href']
    webpage = requests.get(URL2)
    soup2 = BeautifulSoup(webpage.text)
    statement_content = soup2.find_all('article',attrs={'class':'m-textblock'})
    content.append(''.join(statement_content[0].text.strip().splitlines()))


  for i in statement_quote:
   atag=i.find_all('a')[0]   

  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  for i in target:
      link4 = i.find('div',attrs={'class':'c-image'}).find('img').get('alt')
      targets.append(link4)

In [ ]:
n = 6
for i in range(1,n):
  scrape_website(i)

In [ ]:
data = pd.DataFrame(columns = ['author','statement','article','source','date','target'])
data['author'] = pd.Series(authors)
data['statement']= pd.Series(statements)
data['source']= pd.Series(sources)
data['article'] = pd.Series(content)
data['date']= pd.Series(dates)
data['target']= pd.Series(targets)

data

,author,statement,article,source,date,target
0,Tom Kertscher,"Earthquake in Turkey and Syria is ""a systemic ...",The 7.8-magnitude temblor that killed more tha...,Instagram posts,"February 9, 2023",pants-fire
1,Tom Kertscher,“NASA stopped exploring the ocean.”,"A video shared on Instagram claimed that NASA,...",Instagram posts,"January 30, 2023",pants-fire
2,Maria Ramirez,“The White House is now attempting to ban all ...,Fox News viewers might think a government agen...,Sean Hannity,"• January 12,",false
3,Andy Nguyen,"Large machinery is “required to move 500,000 p...",Electric cars are frequently criticized for hi...,Instagram posts,"December 13, 2022",false
4,Faithlyn Graham,West Virginia is poised to “receive $25M for 1...,"Sen. Joe Manchin, D-W.Va., was one of the lawm...",Joe Manchin,"November 10, 2022",true
...,...,...,...,...,...,...
985,Manuela Tobias,NaN,NaN,NaN,"August 23, 2018",NaN
986,Amy Sherman,NaN,NaN,NaN,"August 22, 2018",NaN
987,Lucia Geng,NaN,NaN,NaN,"August 14, 2018",NaN
988,Chris Nichols,NaN,NaN,NaN,"August 6, 2018",NaN


In [ ]:
data.to_csv('New_Environment.csv')